# Deploying Machine Learning Models using ksonnet and Ambassador
## Experimental


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [ ]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true

## Install Ambassador
See the Abassador [getting started](https://www.getambassador.io/user-guide/getting-started) docs. Eventually, this would also be done via ksonnet.

In [5]:
!kubectl apply -f resources/ambassador-service-minikube.yaml

service "ambassador" created


In [6]:
!kubectl apply -f https://getambassador.io/yaml/ambassador/ambassador-no-rbac.yaml

service "ambassador-admin" created
deployment "ambassador" created


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [7]:
!ks init my-ml-deployment --api-spec=version:v1.8.0

INFO  Using context 'kubeflow' from the kubeconfig file specified at the environment variable $KUBECONFIG
INFO  Creating a new app 'my-ml-deployment' at path '/home/clive/work/seldon/seldon2.0/seldon-core/notebooks/my-ml-deployment'
WARNING !
Kubernetes version v1.8.0 is currently supported as Beta; you may encounter unexpected behavior
INFO  Creating environment 'default' with namespace '', pointing at server at address 'https://192.168.99.100:8443'
INFO  Generating environment metadata at path '/home/clive/work/seldon/seldon2.0/seldon-core/notebooks/my-ml-deployment/environments/default'
INFO  ksonnet app successfully created! Next, try creating a component with `ks generate`.


In [8]:
!cd my-ml-deployment && \
    ks registry add seldon-core github.com/SeldonIO/seldon-core/tree/master/seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --apifeServiceType=NodePort --withApife=false && \
    ks apply default

INFO  Retrieved 12 files
INFO  Writing component at 'components/seldon-core'
INFO  Updating deployments default.seldon-cluster-manager
INFO  Creating non-existent deployments default.seldon-cluster-manager
INFO  Updating deployments default.redis
INFO  Creating non-existent deployments default.redis
INFO  Updating services default.redis
INFO  Creating non-existent services default.redis
INFO  Updating customresourcedefinitions seldondeployments.machinelearning.seldon.io
INFO  Creating non-existent customresourcedefinitions seldondeployments.machinelearning.seldon.io


## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [9]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I./proto --python_out=./proto --grpc_python_out=./proto ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [10]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

MINIKUBE_IP=commands.getoutput('minikube ip')

def rest_request(deploymentName):
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30033/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=payload)
    print response.status_code
    print response.text   
    
    
def rest_request_auth(deploymentName,username,password):
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30033/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=payload,
                auth=HTTPBasicAuth(username, password))
    print response.status_code
    print response.text   

def grpc_request(deploymentName):
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('seldon',deploymentName)]
    response = stub.Predict(request=request,metadata=metadata)
    print response

## Create Seldon Deployment

**Check everything is running before continuing**

In [20]:
!kubectl get pods

NAME                                                   READY     STATUS    RESTARTS   AGE
ambassador-64d5c4d75f-5nz8h                            2/2       Running   0          4m
ambassador-64d5c4d75f-bnwkr                            2/2       Running   0          4m
ambassador-64d5c4d75f-wl854                            2/2       Running   0          4m
redis-5767447797-fwt6w                                 1/1       Running   0          3m
seldon-cluster-manager-5b9dc4d987-mgdpx                1/1       Running   0          3m
test-deployment-fx-market-predictor-5779f7794b-pw8zw   2/2       Running   0          2m


In [13]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [21]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[replicasAvailable:1 name:test-deployment-fx-market-predictor replicas:1]]]

## Get predictions

#### REST Request

In [22]:
rest_request("seldon-deployment-example")

200
{
  "meta": {
    "puid": "g5t3kjsutm3iho5mlui2a5kopf",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [23]:
grpc_request("seldon-deployment-example")

meta {
  puid: "b93boe9fdr3l2t6dfc6h3pbb27"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [24]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml

service "example-auth" created
deployment "example-auth" created


** Need to wait until running before adding Ambassador config **

In [27]:
!kubectl get pods

NAME                                                   READY     STATUS    RESTARTS   AGE
ambassador-64d5c4d75f-5nz8h                            2/2       Running   0          5m
ambassador-64d5c4d75f-bnwkr                            2/2       Running   0          5m
ambassador-64d5c4d75f-wl854                            2/2       Running   0          5m
example-auth-f6588c64d-8bnlj                           1/1       Running   0          18s
redis-5767447797-fwt6w                                 1/1       Running   0          4m
seldon-cluster-manager-5b9dc4d987-mgdpx                1/1       Running   0          4m
test-deployment-fx-market-predictor-5779f7794b-pw8zw   2/2       Running   0          2m


In [28]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml

service "example-auth" configured


Show failed request when auth is running

In [29]:
rest_request("seldon-deployment-example")

401



Show successful request with auth

In [30]:
rest_request_auth("seldon-deployment-example","username","password")

200
{
  "meta": {
    "puid": "on7231f70bhejfu2db4gg0p8hd",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


# Tear down

In [31]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml

service "example-auth" deleted
deployment "example-auth" deleted


In [32]:
!kubectl delete -f https://getambassador.io/yaml/ambassador/ambassador-no-rbac.yaml

service "ambassador-admin" deleted
deployment "ambassador" deleted


In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment